# CosmicDS Survey Data Analysis
_Some documentation_ **Courtesy of Copilot**

This notebook processes pre- and post-survey data for CosmicDS. It loads, cleans, merges, and summarizes survey responses.




# Guidance for Exploring Data, Relations, and Correlations in This Notebook

This notebook is designed to help you analyze and explore pre- and post-survey data from the CosmicDS project. Below are some tips and guidance for using the notebook to explore relationships and correlations in your data.

---

## 1. **Understanding the Data Structure**

- **DataFrames:** The main data is stored in pandas DataFrames (`df_pre`, `df_post`, `df_combined`).
- **Questions Metadata:** The `questions` DataFrame contains metadata about each survey question, including type (Likert, content, demographic, etc.).
- **Merged Data:** `df_combined` merges pre- and post-survey responses, along with class and educator info.

---

## 2. **Exploring Survey Responses**

- **Group By:** Use `groupby` to aggregate responses by class, educator, question, or demographic group.
    ```python
    # Example: Group by class and question
    df_combined.groupby(['class_name', 'question'])['response_post'].value_counts()
    ```
- **Filtering:** Filter DataFrames to focus on specific questions, classes, or student groups.
    ```python
    # Example: Filter for a specific question
    df_combined[df_combined['question'] == 'Your Question Text']
    ```


In [ ]:
import pandas as pd
import numpy as np
from astropy.table import Table
import matplotlib.pyplot as plt
from importlib import reload


import os

from importlib import reload

from IPython.display import display, HTML


def pprint(pandas_dataframe: pd.DataFrame, max_width=1000, **kwargs):
    return HTML(
        "\n".join(
            Table.from_pandas(pandas_dataframe).pformat(max_width=max_width, **kwargs)
        )
    )


import class_analytics_utils as cau

reload(cau)
pd.set_option("display.max_colwidth", None)

USE_FRESH_DATA = True

## Data Loading and Setup
- Loads survey data from local CSV or API.
- Cleans up question descriptions and aligns columns between pre and post.
- Drops unnecessary columns.
- Uses pandas DataFrames for all tabular data.

In [ ]:
from qualtrics_keys import (
    survey_id_2025_post,
    survey_id_2025_pre,
    survey_id_2024_post,
    survey_id_2024_pre,
)


id_column = "Intro information_1"

# pre_response = cau.load_pre_data(from_api=False)
file_exists = os.path.exists("2025_pre_response.csv")

pre_response = cau.load_data(
    from_api=USE_FRESH_DATA,
    survey_id=survey_id_2025_pre,
    filename="2025_pre_response.csv",
)
header_pre, description_pre, data_pre = cau.parse_response(pre_response)


post_response = cau.load_data(
    from_api=USE_FRESH_DATA,
    survey_id=survey_id_2025_post,
    filename="2025_post_response.csv",
)
header_post, description_post, data_post = cau.parse_response(post_response)


# updates some columns to match properly, and drop columns in the list
drop_columns = ["CosmicDS Pre-Survey - Click to write Form Field 4"]
cau.column_cleanup(
    header_pre,
    description_pre,
    data_pre,
    header_post,
    description_post,
    data_post,
    drop_columns=drop_columns,
)

for question in description_post:
    # makes question tags match between matching questions
    # probably don't need to do this since we do everything based on the question text
    cau.get_pre_header_for_post_question(
        question,
        header_pre,
        description_pre,
        header_post,
        description_post,
        verbose=False,
    )


# print out the number of records
print("\n\n\n Number of records in Pre/Post survey")
print("Pre-survey records: ", len(data_pre))
print("Post-survey records: ", len(data_post))

USE_FRESH_DATA = False # set to false to use the data from the last run

## DataFrame Construction and Cleaning
- Converts wide-format survey data to long format using `pd.melt`.
- Merges question metadata.
- Processes and combines demographic and reflection questions.
- Converts Likert responses to numeric for analysis.

In [ ]:
try:
    import qualtrics_data_fixes as qdf
    # qualtrics_data_fixes contains two functions to modify the data
    # These functions should not be uploaded to the repo

    reload(qdf)
    qdf.modify_pre_data(data_pre)
    qdf.modify_post_data(data_post)
except ImportError:
    print("No qualtrics_data_fixes.py file found.  Applying no fixes.")
    pass

In [ ]:
reload(cau)
id_column = description_post[header_post.index("Intro information_1")]
instructor_column = "CosmicDS Pre-Survey - Instructor's Last Name"
section_column = 'CosmicDS Pre-Survey - Course/Section'


df_pre, df_post = cau.create_initial_dataframes(
    header_pre,
    description_pre,
    data_pre,
    header_post,
    description_post,
    data_post,
    id_column,
)
assert df_pre[id_column].is_unique, "Duplicate values found in id_column of df_pre"
assert df_post[id_column].is_unique, "Duplicate values found in id_column of df_post"


def process_likert_columns(df, questions):
    # convert the likert columns to numeric
    for col in df.columns:
        if col in questions["question"].values:
            if questions.loc[questions["question"] == col]["is_likert"].values[0]:
                df[col] = df[col].apply(lambda x: cau.convert_likert_to_numeric(x))
    return df


pre_column_filter = lambda arr: [p for p in arr if (p in df_pre.columns)]
post_column_filter = lambda arr: [p for p in arr if (p in df_post.columns)]


def drop_columns(columns):
    df_pre.drop(columns=pre_column_filter(columns), inplace=True)
    df_post.drop(columns=post_column_filter(columns), inplace=True)


questions = cau.create_question_dataframe(
    df_pre, df_post, header_pre, description_pre, header_post, description_post
)
cau.add_likert_columns(questions, df_pre, df_post)
df_pre = process_likert_columns(df_pre, questions)
df_post = process_likert_columns(df_post, questions)

removed_questions = []

questions, removed = cau.process_parent_questions(questions, df_pre, df_post)
removed_questions.extend(removed)
drop_columns(removed_questions)

questions, removed = cau.process_race_questions(questions, df_pre, df_post)
removed_questions.extend(removed)
drop_columns(removed_questions)

questions, removed = cau.process_gender_questions(questions, df_pre, df_post)
removed_questions.extend(removed)
drop_columns(removed_questions)

questions, removed = cau.process_school_questions(questions, df_pre, df_post)
removed_questions.extend(removed)
drop_columns(removed_questions)

questions, removed = cau.process_esl_questions(questions, df_pre, df_post)
removed_questions.extend(removed)
drop_columns(removed_questions)

questions, removed = cau.process_activity_questions(questions, df_pre, df_post)
removed_questions.extend(removed)
drop_columns(removed_questions)

questions, removed = cau.process_confused_questions(questions, df_pre, df_post)
removed_questions.extend(removed)
drop_columns(removed_questions)

cau.add_content_question_meta(questions, description_pre, description_post)
cau.add_likert_meta(questions)

# Here we are going to add in the meta data for the questions
# While df_pre and df_post have the questions as columns
# this step performs a "melt" taking the wide format and making it long
# So that each student entry now has a row for each question & it's response
# the responses are under the column "response" and the question is under "question"
id_columns = list(questions.groupby("question_category").get_group("intro")["question"])


pre_only_questions = [
    "Parent 1 Education",
    "Parent 1 Gender",
    "Parent 2 Education",
    "Parent 2 Gender",
    "Student Race",
    "Student Gender",
    "Student School Level",
    "Student ESL",
]

df_pre_merged, df_post_merged = cau.merge_questions_into_dataframes(
    df_pre, df_post, questions, id_column, id_columns, pre_only_questions
)

## Matching and Merging Pre/Post Data
- Matches students by ID between pre and post surveys.
- Merges responses on ID and question.
- Adds class and educator info from the database.
- Calculates response rates and basic stats.

In [ ]:
print("Pre-test merged records: ", len(df_pre_merged))
# display(pprint(df_pre_merged.head(5), html=True))
print("Post-test merged records: ", len(df_post_merged))
# display(pprint(df_post_merged.head(5), html=True))

In [ ]:
# This line counts the number of questions in each (question_category, group)
# combination by exploding the 'groups' column and grouping by 'question_category' and 'groups'.
print("Number of questions in each question qategory and group:")
questions.explode("groups").groupby(["question_category", "groups"]).count().rename(
    {"question": "count"}, axis=1
)["count"]

In [ ]:
reload(cau)
matched_ids = list(
    set(df_pre_merged[id_column]).intersection(df_post_merged[id_column])
)
orphraned_pre = list(
    set(df_pre_merged[id_column]) - set(df_post_merged[id_column])
)
orphraned_post = list(
    set(df_post_merged[id_column]) - set(df_pre_merged[id_column])
)

print("Number of orphaned pre ids:", len(orphraned_pre))
print("Number of orphaned post ids:", len(orphraned_post))

matched_idx = df_pre_merged[df_pre_merged[id_column].isin(matched_ids)].index
matched_idx_post = df_post_merged[df_post_merged[id_column].isin(matched_ids)].index
print("Number of matched ids:", len(matched_ids))

# merging on the id_column and the questions
# By doing an outer join,
df_combined = pd.merge(
    df_pre_merged,
    df_post_merged,
    on=[id_column] + questions.columns.tolist(),
    how="outer",  # inner join to keep only matching rows
    suffixes=("_pre", "_post"),
)

def notnull(value):
    return value not in ['(empty)', '', None, np.nan]
def has_pre_and_post_respose(row):
    return notnull(row["response_pre"]) and notnull(row["response_post"])

# matched_ids = (df_combined
#                .drop_duplicates(subset=[id_column])
#                .apply(lambda row: has_pre_and_post_respose(row), axis=1)
# )


df_combined["matched"] = df_combined[id_column].apply(
    lambda x: x in matched_ids
)

df_combined["orphaned_pre"] = df_combined[id_column].apply(
    lambda x: x in orphraned_pre
)
df_combined["orphaned_post"] = df_combined[id_column].apply(
    lambda x: x in orphraned_post
)

c = list(
    [id_column] + pre_only_questions
    + ['matched', 'orphaned_pre', 'orphaned_post']
    + questions.columns.tolist()
    + ["response_pre", "correct_pre", "response_post", "correct_post"]
    + [c + "_pre" for c in id_columns[1:]]
    + [c + "_post" for c in id_columns[1:]]
)

df_combined = df_combined[c]
# df_combined = df_combined.rename(columns={"question": "question"})


# get the appropriate classroom data
df_combined = cau.get_class_info(df_combined, id_column)

print(f"There are {len(df_combined)} rows in the combined dataframe")
print(f"There are {len(df_combined[df_combined['matched']])} rows with matched ids")

In [ ]:
# this will print out all of the students for examination
df_combined[['CosmicDS Pre-Survey - CosmicDS Student ID number (Ask your instructor if you do not know this)',
            "CosmicDS Pre-Survey - Instructor's Last Name_pre",
       'CosmicDS Pre-Survey - Course/Section_pre',
       "CosmicDS Pre-Survey - Instructor's Last Name_post",
       'CosmicDS Pre-Survey - Course/Section_post', 'student_id', 'class_name',
       'class_id', 'Educator']].head(10) #.to_excel('studentsclassroom.xlsx', index=False, engine="openpyxl")

In [ ]:
# some basic stats
print(f"Unique IDs in combined: {len(df_combined[id_column].unique())}")
print(f"Unique IDs in pre-survey: {len(df_pre[id_column].unique())}")
print(f"Unique IDs in post-survey: {len(df_post[id_column].unique())}")
print(f"Unique IDs in merged pre-survey: {len(df_pre_merged[id_column].unique())}")
print(f"Unique IDs in merged post-survey: {len(df_post_merged[id_column].unique())}")

missing_in_pre = set(df_post_merged[id_column]) - set(df_pre_merged[id_column])
missing_in_post = set(df_pre_merged[id_column]) - set(df_post_merged[id_column])

print(f"IDs in post but not in pre: {len(missing_in_pre)}")
print(f"IDs in pre but not in post: {len(missing_in_post)}")
print("List of IDs in post but not in pre:", missing_in_pre)
print("List of IDs in pre but not in post:", missing_in_post)

In [ ]:
# Create a 1 row per student view of the data for explorting to excel
questions_both = df_combined.pivot(
    index=id_column,
    columns=["question"],
    values=["response_pre", "response_post"],
    # aggfunc="first"
)
questions_both.columns = ['++'.join(map(str, col)).strip() for col in questions_both.columns]

# Merge back the rest of the non-pre/post q columns (demographics, etc)
questions_both = pd.merge(
    df_combined.drop_duplicates(subset=[id_column]).set_index(id_column),
    questions_both,
    on=id_column,
    how="left"
)
# reset ++ columns to tuples
questions_both.columns = [
    tuple(col.split("++")) if "++" in col else (col, "") for col in questions_both.columns
]
# # multi-index columns where appropriate
questions_both.columns = pd.MultiIndex.from_tuples([
    (col, '') if not isinstance(col, tuple) else col  # promote strings to (col, '')
    for col in questions_both.columns
])

questions_both.to_excel(
    "2025_combined_questions_row.xlsx",
    index=True,
    # engine="openpyxl",
    sheet_name="2025_combined"
)

In [ ]:
print("Response Summary")
response_summary = (
    df_combined.groupby("both")
    .get_group(True)
    .drop_duplicates(subset=[id_column])
    .groupby(["class_name", "Educator", "class_id"])
    .agg(
        {
            "response_pre": lambda x: x.notnull().sum(),
            "response_post": lambda x: x.notnull().sum(),
            "matched": lambda x: x.sum(),
            "orphaned_pre": lambda x: x.sum(),
            "orphaned_post": lambda x: x.sum(),
        }
    )
    .assign(post_response_rate=lambda df: df["matched"] / (df["response_pre"]))
    .reset_index()
)

print(f"Overall response rate: {response_summary['post_response_rate'].mean() * 100:.2f}%")
response_summary

In [ ]:
print("Matched totals")
(df_combined
 .groupby('Educator')
 .apply(
    lambda group: group.drop_duplicates(subset=[id_column])["matched"].sum()
    )
    )

In [ ]:
print("Class Summary for Content (Science Questions)")
print("# of of indivual questions or percentage which are correct")
class_score_summary = (
    df_combined.groupby("both")
    .get_group(True)
    .groupby(["question_category"]).get_group("science")
    .groupby(["class_name", "Educator", "class_id"])
    .agg(
        {
            "response_pre": lambda x: x.notnull().sum(),
            "response_post": lambda x: x.notnull().sum(),
            "matched": lambda x: x.sum(),
            "correct_pre": lambda x: f"{x.sum() / x.notnull().sum():.0%}",
            "correct_post": lambda x: f"{x.sum() / x.notnull().sum():.0%}",
        }
    )
    .assign(post_response_rate=lambda df: df["matched"] / (df["response_pre"]))
    .reset_index()
)


# Define styles for pandas formatting
def style_class_score_summary(row):
    # (modified from) Generated by Copilot
    if row['response_post'] == 0:
        return ['background-color: #FFCCCC; color: black'] * len(row)  # Pale blue for no post
    elif float(row['correct_post'].strip('%')) / 100 < 0.35:
        return ['background-color: #F00' if col == 'correct_post' else '' for col in row.index]  # Pale red for < 30%
    elif float(row['correct_post'].strip('%')) / 100 > 0.75:
        return ['background-color: #CCFFCC; color: black' if col == 'correct_post' else '' for col in row.index]  # Pale green for > 80%
    else:
        return [''] * len(row)

# Apply the styles
styled_class_score_summary = class_score_summary.style.apply(style_class_score_summary, axis=1)

styled_class_score_summary


In [ ]:
by_teacher = df_combined.groupby("Educator")
by_class = df_combined.groupby("class_name")


by_teacher_class = df_combined.groupby(["Educator", "class_name"])
print("Teacher and Class Groups")
for name, group in by_teacher_class:
    print(f"Group: {name}")

# Selecting a Group

```python
# To create a group, you can use the `groupby` method on the DataFrame.
# For example, to group by "Educator":
grouped = df_combined.groupby("Educator")

# To select a specific group, use the `get_group` method:
selected_group = grouped.get_group("Educator Name")

# To view the available keys in the group:
print("Available group keys:", grouped.groups.keys())
```

For a specifier which could have multiple values, like race or gender, we need to "explode" the identifier. (This means students can be counted in multiple groups). For example

```python
by_race = df_combined.explode("Student Race").groupby("Student Race")
print(by_race.groups.keys()) # view valid keys
```

You can even group at multiple levels, so by Teacher and Class
```python
by_teacher_class = df_combined.groupby(["Educator", "class_name"])
```

## Valid columns for selecting by
- 'CosmicDS Pre-Survey - CosmicDS Student ID number (Ask your instructor if you do not know this)' (can select it with `id_column`)
- Parent Demographics
    - 'Parent 1 Education'
    - 'Parent 1 Gender'
    - 'Parent 2 Education'
    - 'Parent 2 Gender'
- Student Demographics
    - 'Student Race'
    - 'Student Gender'
    - 'Student School Level'
    - 'Student ESL'
- 'matched'
    - Only pre-post matched students
    - Only use `get_group(True)`, `False` returns nonsense
- 'orphaned_pre'
- 'orphaned_post'
- 'question'
    - used for applying
- Classroom info
    - 'class_name'
    - 'class_id'
    - 'Educator'

In [ ]:
# To select only matched responses, use only the matched group
# unmatched post-feedback will not be included
selected_group = df_combined.groupby("matched").get_group(True) 

# To selection an specific educator
# selected_group = df_combined.groupby("Educator").get_group(("Educator Name"))

# groups = df_combined.explode("Student Race").groupby("Student Race")

# print("Possible Groups:", list(groups.groups.keys()),'\n')
# selected_group  = groups.get_group("Black or African American")

# no group
# selected_group = by_teacher.get_group(("Educator Names"))

Note on effect sizes for binary data (the content group)

| **#** | **Name**                               | **Formula / Description**                                                                   | **Proper Use**                                         | **Pairing Assumed?** |
| ----- | -------------------------------------- | ------------------------------------------------------------------------------------------- | ------------------------------------------------------ | -------------------- |
| 1     | **Cohen’s *h***                        | $h = 2 \cdot \arcsin(\sqrt{p_2}) - 2 \cdot \arcsin(\sqrt{p_1})$                             | Standardized effect size for proportions               | ❌ No                 |
| 2     | **Standardized Prop. Difference**      | $z = \frac{p_2 - p_1}{\sqrt{ \frac{p_1(1-p_1)}{n_1} + \frac{p_2(1-p_2)}{n_2} }}$            | Signal-to-noise effect size or z-test                  | ❌ No                 |
| 3     | **Two-sample z-test for proportions**  | $z = \frac{p_2 - p_1}{\sqrt{P(1-P)(1/n_1 + 1/n_2)}}, \quad P = \frac{c_1 + c_2}{n_1 + n_2}$ | Hypothesis test for group proportion difference        | ❌ No                 |
| 4     | **Binary Cohen’s *d*** *(your method)* | $d = \frac{p_2 - p_1}{\sqrt{ \text{pooled } \left( \frac{p(1 - p)}{n} \right) }}$           | Cohen-style standardized binary ES                     | ❌ No                 |
| 5     | **McNemar’s Test**  (one-sided improvement)        | $p = \text{binomtest}(k = \text{True→False},\ n = \text{True→False} + \text{False→True},\ p = 0.5, \text{alt=greater})$                        | Tests for directional change in **paired** binary data | ✅ Yes                |

The `effect_size` we report is the "Binary Cohen's d", it is a version of the standard Cohen's d, where the variance used is for a binomial distributions. This value will diverge quickly for large N, basically because the variance gets really small

**Cohen's h** is the more _official_ statistic for this comparison, and is meant be interpreted on the same scaling as a typcial cohen's d. it is better behaved, not exploding for small variance

Not listed here is the result of binomtest, which is precise for binary data and small numbers. The p-value and percentage difference are the best measures  b=False→True, c=True→False

The **most precise** statistic is McNemar's


## Note on P-values
- `mcnemar_p` tests **whether more people improved than regressed**
- `p_value` is an overall p-value and only tests **are overall pre/post proportions different?**

## Values returned for multiple choice data

| **Parameter**                                         | **Description**                                                                   |
| ----------------------------------------------------- | --------------------------------------------------------------------------------- |
| `percent_correct_pre`                                 | Percent of students who answered correctly on the pre-test                        |
| `percent_correct_post`                                | Percent of students who answered correctly on the post-test                       |
| `summary`                                             | Summary sentence describing overall change and statistical significance           |
| `effect_size`                                         | Binary Cohen’s *d*-style effect size based on raw count difference                |
| `p_value`                                             | Binomial test p-value for change in total correctness from pre to post            |
| `cohens_h`                                            | Cohen’s *h*: standardized difference in proportions using arcsin-sqrt transform   |
| `mcnemar_statistic`                                   | Proportion of changers who improved (f->t / (f->t + t->f))    |
| `mcnemar_p`                                           | One-sided McNemar p-value: tests if improvements exceed regressions               |
| `discordant_summary`                                  | Summary sentence for direction and significance of paired changes                 |
| `standardized_difference`                             | Z-like standardized difference in proportions using unpooled binomial variance    |
| `z_statistic`                                         | Z-statistic for difference in proportions assuming equal proportions under null   |
| `false_to_true_p_value`                               | Binomial test for students who improved out of those who were initially incorrect |
| `sqrt_pooled_variance`                                | Square root of pooled binomial variance (used in effect size calculation)         |
| `pre_to_post_change`                                  | Net change in number of correct answers (post − pre)                              |
| `pre_to_post_change_percent`                          | Net change as a percent of total students                                         |
| `direction`                                           | Sign of net change: +1 (improvement), 0, or -1 (decline)                          |
| `confused_count`                                      | Number of students who regressed (correct → incorrect)                            |
| `confused`                                            | Percent of previously correct students who got it wrong post-test                 |
| `confused_p_value`                                    | Binomial test for whether regression rate is higher than chance                   |
| `n_discordant`                                        | Number of students who changed responses (correct → incorrect or vice versa)      |
| `n_change_to_true`                                    | Number of students who improved (incorrect → correct)                             |
| `tag_pre`, `tag_post`                                 | Metadata: tags associated with pre and post items                                 |
| `pre_correct`, `post_correct`                         | Count of correct answers pre and post                                             |
| `pre_count`, `post_count`                             | Total number of pre and post responses                                            |
| `group`                                               | Group identifier(s), e.g., demographic label or class group                       |


In [ ]:
reload(cau)

def group_analysis(selected_group):
    questions_only_in_pre = selected_group.groupby(["both", "in_pre", "in_post"]).get_group(
        (False, True, False)
    )
    questions_only_in_post = selected_group.groupby(
        ["both", "in_pre", "in_post"]
    ).get_group((False, False, True))


    drop_questions = [
        "Astro Content MLO 1b",
        "Data Skills MLO 3c",
        "Data Skills likert MLO 3c",
        "Nature of science MLO 2a",
        "STEM interest & Identity MLO 1a",
        "Score",
    ]
    student_info = cau.create_stats_for_pre(questions_only_in_pre, pre_only_questions) # a list of series
    print("Available Items: ", [s.index.name for s in student_info])
    post_reflection_summary = cau.create_post_reflection_summary(questions_only_in_post, id_column)


    # Get the list of matched questions
    # matched = matched rows
    # both = question appears in both pre and post
    try:
        matched_responses = (
            selected_group.groupby(["matched", "both"])
            .get_group((True, True))
            .groupby("question_category")
        )
    except KeyError:
        print("No matched responses found to create matched_responses")
        return None, None, None, None, questions, student_info, post_reflection_summary, None
    print(matched_responses.groups.keys())

    # get the likert questions
    likert_group = matched_responses.get_group("likert")

    # get the multiple choice questions
    content_group = matched_responses.get_group("science")

    # group responses by questions
    likert_questions = likert_group.groupby("question")
    content_questions = content_group.groupby("question")

    # apply our summary to each group/question
    likert_out = likert_questions.apply(cau.create_likert_summary)

    # reset index to be tag_pre and put the current index in 'question'
    likert_out.reset_index(inplace=True)
    likert_out.set_index("tag_pre", inplace=True)
    likert_out.sort_index(key=lambda x: list(map(cau.q_to_num, x)), inplace=True)
    # removing the question pre-amble.
    likert_out["question"] = likert_out["question"].apply(
        lambda x: x.split("? -", maxsplit=1)[-1].strip()
    )

    # apply summary to each class of likert questions
    likert_groups_out = (
        matched_responses.get_group("likert")
        .explode("likert_group")
        .groupby("likert_group")
        .apply(lambda gr: cau.create_likert_summary(gr, force_improved=True))
    )


    content_out = content_questions.apply(cau.create_content_summary)
    content_out.reset_index(inplace=True)
    content_out.set_index("tag_pre", inplace=True)
    content_out.sort_index(key=lambda x: list(map(cau.q_to_num, x)), inplace=True)


    content_group_summary = (
        content_group.explode("groups").groupby("groups").apply(cau.create_content_summary)
    )
    return (
        content_group_summary, 
        likert_groups_out, 
        likert_out, 
        content_out, 
        questions, 
        student_info, 
        post_reflection_summary, 
        matched_responses
    )

## Summarizing and Exporting Results
- Groups responses by question, class, and educator.
- Summarizes Likert and content (multiple choice) questions.
- Writes all summary tables and raw data to an Excel workbook with multiple sheets.
- For post_reflection_summary, each pd.Series is written as a block to a single sheet.

In [ ]:
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill
from openpyxl.styles import Font

def create_workbook(filename = None, overwrite = False, summary = (None, None, None, None, None, None, None, None)):
    if all(
        [x is None for x in summary]
        ):
        raise ValueError("Summary data is None. Please provide summary data.")
    
    content_group_summary, likert_groups_out, likert_out, content_out, questions, student_info, post_reflection_summary, matched_responses = summary
    # create a two sheet workbook with likert_out and content_out
    wb = Workbook()
    wb.remove(wb["Sheet"])
    
    if content_group_summary is not None:
        # create a sheet for content_group_summary
        ws5 = wb.create_sheet("Content Group Summary")
        ws5.title = "Content Group Summary"
        for r in dataframe_to_rows(content_group_summary, index=True, header=True):
            ws5.append(r)

    
    if likert_groups_out is not None:
        ws7 = wb.create_sheet("Likert Group Summary")
        ws7.title = "Likert Group Summary"
        for r in dataframe_to_rows(likert_groups_out, index=True, header=True):
            ws7.append(r)

    
    if likert_out is not None:
        # create a sheet for likert_out
        ws1 = wb.create_sheet("Lickert Summary")
        ws1.title = "Lickert Questions"
        for r in dataframe_to_rows(likert_out, index=False, header=True):
            ws1.append(r)

    if content_out is not None:
        ws2 = wb.create_sheet("Content Questions Summary")
        ws2.title = "Content Questions"
        for r in dataframe_to_rows(content_out, index=False, header=True):
            ws2.append(r)
    
    if matched_responses is not None:
        likert_both = (
            matched_responses.get_group("likert")
            .pivot(
                index=[id_column],
                columns=["tag_pre", "question"],
                values=["response_pre", "response_post"],
            )
            .apply(pd.to_numeric, errors="coerce")
        )

        questions_both = matched_responses.get_group("science").pivot(
            index=[id_column],
            columns=["tag_pre", "question"],
            values=["response_pre", "response_post", "answer"],
        )
        questions_both = cau.add_answer_column(questions_both)

        # create a sheet for likert_both
        ws3 = wb.create_sheet("Lickert Data")
        ws3.title = "Lickert Data"
        for r in dataframe_to_rows(likert_both, index=True, header=True):
            ws3.append(r)


        # create a sheet for questions_both
        ws4 = wb.create_sheet("Content Question Data")
        ws4.title = "Content Question Data"
        for r in dataframe_to_rows(questions_both, index=True, header=True):
            ws4.append(r)


    # create a page for the questions
    ws6 = wb.create_sheet("question")
    ws6.title = "question"
    excel_questions = questions.copy()
    # drop likert_group column
    excel_questions.drop(columns=["likert_group"], inplace=True)
    excel_questions["groups"] = excel_questions["groups"].apply(
        lambda x: ", ".join(list(x)) if len(x) > 0 else None
    )
    for r in dataframe_to_rows(excel_questions, index=False, header=True):
        ws6.append(r)


    ws8 = wb.create_sheet("Full Combined Data")
    ws8.title = "Full Combined Data"
    for r in dataframe_to_rows(df_combined, index=False, header=True):
        # coercer row into string
        ws8.append([str(x) for x in r])


    # student_info
    ws9 = wb.create_sheet("Student Info")
    ws9.title = "Student Info"
    for info_frame in student_info:
        for r in dataframe_to_rows(info_frame, index=True, header=True):
            ws9.append(r)
        ws9.append([])

    # post_reflection_summary
    ws10 = wb.create_sheet("Post Reflection Summary")
    ws10.title = "Post Reflection Summary"
    for series in post_reflection_summary:

        for r in dataframe_to_rows(pd.DataFrame(series), index=True, header=True):
            ws10.append(r)
        # add some empty rows
        ws10.append([])

    
    if filename is None:
        filename = "all_stats_2025.xlsx"
        
    if os.path.exists(filename) and not overwrite:
        raise FileExistsError(
            f"{filename} already exists. Please remove it before running this code or set overwrite=True"
        )

    wb.save(filename)
    print(f"Workbook saved to {filename}")
    return wb

In [ ]:
summaries = group_analysis(df_combined)
wb = create_workbook(
    filename=f"all_data_summary_2025.xlsx",
    overwrite=True,
    summary=summaries
)
print(f"Workbook saved to all_data_summary_2025.xlsx")

In [ ]:
for name, selected_group in df_combined.groupby(["Educator"]):
    summaries = group_analysis(selected_group)
    wb = create_workbook(
        filename=f"{name}_summary_2025.xlsx",
        overwrite=True,
        summary=summaries
    )
    print(f"Workbook saved to {name}_summary_2025.xlsx")
    